# Inferencing on IDD 00074 obstacle appears for more than 0.1 seconds consecutively and saving timestamps/video clips

In [ ]:
import time
import subprocess
from pathlib import Path
import pandas as pd
from ultralytics import YOLO

# ---------------- CONFIG ----------------
IDD_ROOT = Path("../../data/idd_temporal_test_1/00074")
MODEL_PATH = "../../runs/three_country_training/road_defect_binary/yolov8s/weights/best.pt"

OUTPUT_ROOT = Path("../../runs/idd_temporal_time_based_detection/yolov8s/00074")
OUTPUT_ROOT.mkdir(parents=True, exist_ok=True)

IMG_SIZE = 640
CONF = 0.50
IOU = 0.7
DEVICE = 0

FPS_ASSUMED = 30.0                 # used only for timestamps
MIN_PERSISTENCE_SEC = 0.10         # 👈 replaces “3-frame rule”
MIN_EVENT_GAP_SEC = 0.20           # merge nearby detections
# ---------------------------------------

model = YOLO(MODEL_PATH)

# ---------------- GLOBAL METRICS ----------------
global_frame_idx = 0
total_frames = 0
total_time = 0.0

timestamp_records = []
saved_clip_videos = []

print(f"▶ Processing IDD parent folder: {IDD_ROOT.name}")

# ---------------- PROCESS EACH CLIP ----------------
for clip_dir in sorted(IDD_ROOT.iterdir()):
    if not clip_dir.is_dir():
        continue

    clip_name = f"{IDD_ROOT.name}_{clip_dir.name}"
    print(f"\n▶ Clip: {clip_name}")

    active_start_time = None
    last_detected_time = None
    detected_events = []

    start = time.time()

    results = model.predict(
        source=str(clip_dir),
        imgsz=IMG_SIZE,
        conf=CONF,
        iou=IOU,
        stream=True,
        device=DEVICE,
        save=False
    )

    for r in results:
        timestamp = global_frame_idx / FPS_ASSUMED
        detected = r.boxes is not None and len(r.boxes) > 0

        total_frames += 1

        if detected:
            if active_start_time is None:
                active_start_time = timestamp
            last_detected_time = timestamp

        else:
            # detection ended → evaluate persistence
            if active_start_time is not None:
                duration = last_detected_time - active_start_time
                if duration >= MIN_PERSISTENCE_SEC:
                    detected_events.append(
                        (active_start_time, last_detected_time, duration)
                    )
            active_start_time = None
            last_detected_time = None

        global_frame_idx += 1

    # Handle detection that lasts till last frame
    if active_start_time is not None:
        duration = last_detected_time - active_start_time
        if duration >= MIN_PERSISTENCE_SEC:
            detected_events.append(
                (active_start_time, last_detected_time, duration)
            )

    elapsed = time.time() - start
    total_time += elapsed

    if not detected_events:
        print("   ⏭ Ignored (no persistent defect)")
        continue

    print(f"   ✅ {len(detected_events)} defect event(s) detected")

    # ---------- LOG TIMESTAMPS ----------
    for start_t, end_t, dur in detected_events:
        timestamp_records.append({
            "parent_video": IDD_ROOT.name,
            "clip": clip_dir.name,
            "start_time_sec": round(start_t, 3),
            "end_time_sec": round(end_t, 3),
            "duration_sec": round(dur, 3)
        })

    # ---------- SAVE CLIP ----------
    save_results = model.predict(
        source=str(clip_dir),
        imgsz=IMG_SIZE,
        conf=CONF,
        iou=IOU,
        stream=True,
        device=DEVICE,
        save=True,
        project=str(OUTPUT_ROOT),
        name=clip_name
    )

    for _ in save_results:
        pass

    run_dir = OUTPUT_ROOT / clip_name
    mp4_name = "detected.mp4"

    subprocess.run(
        [
            "ffmpeg", "-y",
            "-framerate", str(FPS_ASSUMED),
            "-pattern_type", "glob",
            "-i", "*.jpg",
            "-c:v", "libx264",
            "-pix_fmt", "yuv420p",
            mp4_name
        ],
        cwd=run_dir,
        check=True
    )

    saved_clip_videos.append(run_dir / mp4_name)
    print(f"   🎥 Saved clip video")

# ---------------- MERGE ALL CLIPS ----------------
if saved_clip_videos:
    concat_file = OUTPUT_ROOT / "concat.txt"
    with open(concat_file, "w") as f:
        for v in saved_clip_videos:
            f.write(f"file '{v.resolve()}'\n")

    final_video = OUTPUT_ROOT / "00074_full_detected_video.mp4"

    subprocess.run(
        [
            "ffmpeg", "-y",
            "-f", "concat",
            "-safe", "0",
            "-i", str(concat_file),
            "-c", "copy",
            str(final_video)
        ],
        check=True
    )

    print(f"\n🎬 Merged video saved: {final_video}")

# ---------------- SUMMARY DATAFRAME ----------------
avg_fps = total_frames / total_time if total_time > 0 else 0
video_length_sec = total_frames / FPS_ASSUMED

summary_df = pd.DataFrame([{
    "video_id": IDD_ROOT.name,
    "total_length_sec": round(video_length_sec, 2),
    "processed_frames": total_frames,
    "processed_time_sec": round(total_time, 2),
    "avg_fps": round(avg_fps, 2),
    "num_detected_events": len(timestamp_records)
}])

summary_df.to_csv(OUTPUT_ROOT / "video_summary.csv", index=False)

# ---------------- TIMESTAMP FILE ----------------
timestamps_df = pd.DataFrame(timestamp_records)
timestamps_df.to_csv(OUTPUT_ROOT / "defect_timestamps.csv", index=False)

print("\n📊 Summary:")
print(summary_df)
print(f"\n🕒 Timestamp log saved ({len(timestamp_records)} events)")


In [3]:
# Load data

defect_data_00074 = pd.read_csv(OUTPUT_ROOT / "defect_timestamps.csv")
video_summary_00074 = pd.read_csv(OUTPUT_ROOT / "video_summary.csv")

In [4]:
video_summary_00074.head()

,video_id,total_length_sec,processed_frames,processed_time_sec,avg_fps,num_detected_events
0,74,31.0,930,25.28,36.78,13


In [5]:
defect_data_00074.head()

,parent_video,clip,start_time_sec,end_time_sec,duration_sec
0,74,024235_leftImg8bit,1.033,1.233,0.200
1,74,067826_leftImg8bit,2.967,3.067,0.100
2,74,136623_leftImg8bit,4.533,4.733,0.200
3,74,136623_leftImg8bit,4.800,4.933,0.133
4,74,198202_leftImg8bit,7.367,7.500,0.133


# Inferencing on IDD 00082 obstacle appears for more than 0.1 seconds consecutively and saving timestamps/video clips

In [6]:
import time
import subprocess
from pathlib import Path
import pandas as pd
from ultralytics import YOLO

# ---------------- CONFIG ----------------
IDD_ROOT = Path("../../data/idd_temporal_test_1/00082")
MODEL_PATH = "../../runs/three_country_training/road_defect_binary/yolov8s/weights/best.pt"

OUTPUT_ROOT = Path("../../runs/idd_temporal_time_based_detection/yolov8s/00082")
OUTPUT_ROOT.mkdir(parents=True, exist_ok=True)

IMG_SIZE = 640
CONF = 0.50
IOU = 0.7
DEVICE = 0

FPS_ASSUMED = 30.0                 # used only for timestamps
MIN_PERSISTENCE_SEC = 0.10         # replaces “3-frame rule”
MIN_EVENT_GAP_SEC = 0.20           # merge nearby detections
# ---------------------------------------

model = YOLO(MODEL_PATH)

# ---------------- GLOBAL METRICS ----------------
global_frame_idx = 0
total_frames = 0
total_time = 0.0

timestamp_records = []
saved_clip_videos = []

print(f"▶ Processing IDD parent folder: {IDD_ROOT.name}")

# ---------------- PROCESS EACH CLIP ----------------
for clip_dir in sorted(IDD_ROOT.iterdir()):
    if not clip_dir.is_dir():
        continue

    clip_name = f"{IDD_ROOT.name}_{clip_dir.name}"
    print(f"\n▶ Clip: {clip_name}")

    active_start_time = None
    last_detected_time = None
    detected_events = []

    start = time.time()

    results = model.predict(
        source=str(clip_dir),
        imgsz=IMG_SIZE,
        conf=CONF,
        iou=IOU,
        stream=True,
        device=DEVICE,
        save=False
    )

    for r in results:
        timestamp = global_frame_idx / FPS_ASSUMED
        detected = r.boxes is not None and len(r.boxes) > 0

        total_frames += 1

        if detected:
            if active_start_time is None:
                active_start_time = timestamp
            last_detected_time = timestamp

        else:
            # detection ended → evaluate persistence
            if active_start_time is not None:
                duration = last_detected_time - active_start_time
                if duration >= MIN_PERSISTENCE_SEC:
                    detected_events.append(
                        (active_start_time, last_detected_time, duration)
                    )
            active_start_time = None
            last_detected_time = None

        global_frame_idx += 1

    # Handle detection that lasts till last frame
    if active_start_time is not None:
        duration = last_detected_time - active_start_time
        if duration >= MIN_PERSISTENCE_SEC:
            detected_events.append(
                (active_start_time, last_detected_time, duration)
            )

    elapsed = time.time() - start
    total_time += elapsed

    if not detected_events:
        print("   ⏭ Ignored (no persistent defect)")
        continue

    print(f"   ✅ {len(detected_events)} defect event(s) detected")

    # ---------- LOG TIMESTAMPS ----------
    for start_t, end_t, dur in detected_events:
        timestamp_records.append({
            "parent_video": IDD_ROOT.name,
            "clip": clip_dir.name,
            "start_time_sec": round(start_t, 3),
            "end_time_sec": round(end_t, 3),
            "duration_sec": round(dur, 3)
        })

    # ---------- SAVE CLIP ----------
    save_results = model.predict(
        source=str(clip_dir),
        imgsz=IMG_SIZE,
        conf=CONF,
        iou=IOU,
        stream=True,
        device=DEVICE,
        save=True,
        project=str(OUTPUT_ROOT),
        name=clip_name
    )

    for _ in save_results:
        pass

    run_dir = OUTPUT_ROOT / clip_name
    mp4_name = "detected.mp4"

    subprocess.run(
        [
            "ffmpeg", "-y",
            "-framerate", str(FPS_ASSUMED),
            "-pattern_type", "glob",
            "-i", "*.jpg",
            "-c:v", "libx264",
            "-pix_fmt", "yuv420p",
            mp4_name
        ],
        cwd=run_dir,
        check=True
    )

    saved_clip_videos.append(run_dir / mp4_name)
    print(f"   🎥 Saved clip video")

# ---------------- MERGE ALL CLIPS ----------------
if saved_clip_videos:
    concat_file = OUTPUT_ROOT / "concat.txt"
    with open(concat_file, "w") as f:
        for v in saved_clip_videos:
            f.write(f"file '{v.resolve()}'\n")

    final_video = OUTPUT_ROOT / "00082_full_detected_video.mp4"

    subprocess.run(
        [
            "ffmpeg", "-y",
            "-f", "concat",
            "-safe", "0",
            "-i", str(concat_file),
            "-c", "copy",
            str(final_video)
        ],
        check=True
    )

    print(f"\n🎬 Merged video saved: {final_video}")

# ---------------- SUMMARY DATAFRAME ----------------
avg_fps = total_frames / total_time if total_time > 0 else 0
video_length_sec = total_frames / FPS_ASSUMED

summary_df = pd.DataFrame([{
    "video_id": IDD_ROOT.name,
    "total_length_sec": round(video_length_sec, 2),
    "processed_frames": total_frames,
    "processed_time_sec": round(total_time, 2),
    "avg_fps": round(avg_fps, 2),
    "num_detected_events": len(timestamp_records)
}])

summary_df.to_csv(OUTPUT_ROOT / "video_summary.csv", index=False)

# ---------------- TIMESTAMP FILE ----------------
timestamps_df = pd.DataFrame(timestamp_records)
timestamps_df.to_csv(OUTPUT_ROOT / "defect_timestamps.csv", index=False)

print("\n📊 Summary:")
print(summary_df)
print(f"\n🕒 Timestamp log saved ({len(timestamp_records)} events)")


▶ Processing IDD parent folder: 00082

▶ Clip: 00082_877981_leftImg8bit

image 1/31 /home/saber/GitHub/road_anomaly_detection/src/three_countries/../../data/idd_temporal_test_1/00082/877981_leftImg8bit/0001059.jpeg: 384x640 (no detections), 6.9ms
image 2/31 /home/saber/GitHub/road_anomaly_detection/src/three_countries/../../data/idd_temporal_test_1/00082/877981_leftImg8bit/0001060.jpeg: 384x640 (no detections), 6.6ms
image 3/31 /home/saber/GitHub/road_anomaly_detection/src/three_countries/../../data/idd_temporal_test_1/00082/877981_leftImg8bit/0001061.jpeg: 384x640 (no detections), 6.3ms
image 4/31 /home/saber/GitHub/road_anomaly_detection/src/three_countries/../../data/idd_temporal_test_1/00082/877981_leftImg8bit/0001062.jpeg: 384x640 (no detections), 6.2ms
image 5/31 /home/saber/GitHub/road_anomaly_detection/src/three_countries/../../data/idd_temporal_test_1/00082/877981_leftImg8bit/0001063.jpeg: 384x640 (no detections), 6.4ms
image 6/31 /home/saber/GitHub/road_anomaly_detection/src/

# Inferencing on IDD 00061 obstacle appears for more than 0.1 seconds consecutively and saving timestamps/video clips

In [ ]:
import time
import subprocess
from pathlib import Path
import pandas as pd
from ultralytics import YOLO

# ---------------- CONFIG ----------------
IDD_ROOT = Path("../../data/idd_temporal_test_1/00061")
MODEL_PATH = "../../runs/three_country_training/road_defect_binary/yolov8s/weights/best.pt"

OUTPUT_ROOT = Path("../../runs/idd_temporal_time_based_detection/yolov8s/00061")
OUTPUT_ROOT.mkdir(parents=True, exist_ok=True)

IMG_SIZE = 640
CONF = 0.50
IOU = 0.7
DEVICE = 0

FPS_ASSUMED = 30.0                 # used only for timestamps
MIN_PERSISTENCE_SEC = 0.10         # 👈 replaces “3-frame rule”
MIN_EVENT_GAP_SEC = 0.20           # merge nearby detections
# ---------------------------------------

model = YOLO(MODEL_PATH)

# ---------------- GLOBAL METRICS ----------------
global_frame_idx = 0
total_frames = 0
total_time = 0.0

timestamp_records = []
saved_clip_videos = []

print(f"▶ Processing IDD parent folder: {IDD_ROOT.name}")

# ---------------- PROCESS EACH CLIP ----------------
for clip_dir in sorted(IDD_ROOT.iterdir()):
    if not clip_dir.is_dir():
        continue

    clip_name = f"{IDD_ROOT.name}_{clip_dir.name}"
    print(f"\n▶ Clip: {clip_name}")

    active_start_time = None
    last_detected_time = None
    detected_events = []

    start = time.time()

    results = model.predict(
        source=str(clip_dir),
        imgsz=IMG_SIZE,
        conf=CONF,
        iou=IOU,
        stream=True,
        device=DEVICE,
        save=False
    )

    for r in results:
        timestamp = global_frame_idx / FPS_ASSUMED
        detected = r.boxes is not None and len(r.boxes) > 0

        total_frames += 1

        if detected:
            if active_start_time is None:
                active_start_time = timestamp
            last_detected_time = timestamp

        else:
            # detection ended → evaluate persistence
            if active_start_time is not None:
                duration = last_detected_time - active_start_time
                if duration >= MIN_PERSISTENCE_SEC:
                    detected_events.append(
                        (active_start_time, last_detected_time, duration)
                    )
            active_start_time = None
            last_detected_time = None

        global_frame_idx += 1

    # Handle detection that lasts till last frame
    if active_start_time is not None:
        duration = last_detected_time - active_start_time
        if duration >= MIN_PERSISTENCE_SEC:
            detected_events.append(
                (active_start_time, last_detected_time, duration)
            )

    elapsed = time.time() - start
    total_time += elapsed

    if not detected_events:
        print("   ⏭ Ignored (no persistent defect)")
        continue

    print(f"   ✅ {len(detected_events)} defect event(s) detected")

    # ---------- LOG TIMESTAMPS ----------
    for start_t, end_t, dur in detected_events:
        timestamp_records.append({
            "parent_video": IDD_ROOT.name,
            "clip": clip_dir.name,
            "start_time_sec": round(start_t, 3),
            "end_time_sec": round(end_t, 3),
            "duration_sec": round(dur, 3)
        })

    # ---------- SAVE CLIP ----------
    save_results = model.predict(
        source=str(clip_dir),
        imgsz=IMG_SIZE,
        conf=CONF,
        iou=IOU,
        stream=True,
        device=DEVICE,
        save=True,
        project=str(OUTPUT_ROOT),
        name=clip_name
    )

    for _ in save_results:
        pass

    run_dir = OUTPUT_ROOT / clip_name
    mp4_name = "detected.mp4"

    subprocess.run(
        [
            "ffmpeg", "-y",
            "-framerate", str(FPS_ASSUMED),
            "-pattern_type", "glob",
            "-i", "*.jpg",
            "-c:v", "libx264",
            "-pix_fmt", "yuv420p",
            mp4_name
        ],
        cwd=run_dir,
        check=True
    )

    saved_clip_videos.append(run_dir / mp4_name)
    print(f"   🎥 Saved clip video")

# ---------------- MERGE ALL CLIPS ----------------
if saved_clip_videos:
    concat_file = OUTPUT_ROOT / "concat.txt"
    with open(concat_file, "w") as f:
        for v in saved_clip_videos:
            f.write(f"file '{v.resolve()}'\n")

    final_video = OUTPUT_ROOT / "00061_full_detected_video.mp4"

    subprocess.run(
        [
            "ffmpeg", "-y",
            "-f", "concat",
            "-safe", "0",
            "-i", str(concat_file),
            "-c", "copy",
            str(final_video)
        ],
        check=True
    )

    print(f"\n🎬 Merged video saved: {final_video}")

# ---------------- SUMMARY DATAFRAME ----------------
avg_fps = total_frames / total_time if total_time > 0 else 0
video_length_sec = total_frames / FPS_ASSUMED

summary_df = pd.DataFrame([{
    "video_id": IDD_ROOT.name,
    "total_length_sec": round(video_length_sec, 2),
    "processed_frames": total_frames,
    "processed_time_sec": round(total_time, 2),
    "avg_fps": round(avg_fps, 2),
    "num_detected_events": len(timestamp_records)
}])

summary_df.to_csv(OUTPUT_ROOT / "video_summary.csv", index=False)

# ---------------- TIMESTAMP FILE ----------------
timestamps_df = pd.DataFrame(timestamp_records)
timestamps_df.to_csv(OUTPUT_ROOT / "defect_timestamps.csv", index=False)

print("\n📊 Summary:")
print(summary_df)
print(f"\n🕒 Timestamp log saved ({len(timestamp_records)} events)")


In [8]:
# Loading data saved

defect_data_00061 = pd.read_csv(OUTPUT_ROOT / "defect_timestamps.csv")
video_summary_00061 = pd.read_csv(OUTPUT_ROOT / "video_summary.csv")

In [9]:
defect_data_00061.head()

,parent_video,clip,start_time_sec,end_time_sec,duration_sec
0,61,000160_leftImg8bit,0.0,0.100,0.100
1,61,000160_leftImg8bit,0.2,0.467,0.267


In [10]:
video_summary_00061.head()

,video_id,total_length_sec,processed_frames,processed_time_sec,avg_fps,num_detected_events
0,61,13.13,394,12.13,32.48,2
